In [290]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
from sklearn.metrics import jaccard_score
import pandas as pd
import gensim
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

In [291]:
import gensim
import gensim.downloader as download_api
russian_model = download_api.load('word2vec-ruscorpora-300')
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
import nltk
from nltk.tokenize import word_tokenize
!pip install pymorphy2 > None
import numpy as np
import pandas as pd
import matplotlib as plt
import re
import pandas as pd
import pymorphy2
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import word_tokenize
import nltk
from string import punctuation
nltk.download('stopwords')
from  sklearn.feature_extraction.text import TfidfVectorizer
from  sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
from nltk import wordpunct_tokenize, WordNetLemmatizer, sent_tokenize, pos_tag
from nltk.corpus import stopwords as sw, wordnet as wn
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import string
import re
punctuations = list(punctuation)
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()
import nltk
nltk.download('punkt')
nltk.download('punkt')
tfidf_encoder = TfidfVectorizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Victoria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Victoria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Victoria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Victoria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [292]:
file_path = "df_vacancies2.xlsx"
df_vacancies = pd.read_excel(file_path)
df_vacancies.head(2)

,id_vacancy,название вакансии,дата публикации,компания,уровень зарплаты,URL,описание вакансии,адрес,тип занятости,описание вакансии_tokenized,...,Low_salary,High_salary,Type_employee,Work_time,experience,Duties_unch,Skills_unch,описание_unch,Demands_unch,company_sphere
0,1,Инженер-конструктор,24 ноября 2023 в Санкт-Петербурге,ООО Русь-Турбо,от 70 000 до 120 000 ₽ на руки,https://murmansk.hh.ru/vacancy/87418369?from=v...,Обязанности: Разработка и проектирование запча...,Санкт-Петербург,"Полная занятость, полный день","['обязанность', 'разработка', 'проектирование'...",...,70000,70000,Полная занятость,полный день,1.0,запчасть оснастка приспособление паровой турби...,3d ansys самоорганизация дисциплина коммуникаб...,обязанность разработка проектирование запчасть...,выпуск конструкторский 3d ansys самоорганизаци...,Инженерно-техническая
1,2,Инженер-конструктор,22 ноября 2023 в Санкт-Петербурге,ГРУППА КОМПАНИЙ БАЗИС,от 80 000 до 110 000 ₽ на руки,https://murmansk.hh.ru/vacancy/88536853?from=v...,Группа компаний Базис — крупная многопрофильна...,Санкт-Петербург,"Полная занятость, полный день","['группа', 'компания', 'базис', '—', 'крупный'...",...,80000,80000,Полная занятость,полный день,1.0,довести идеальный конструктивный стадия здание...,NaN,группа компания базис — крупный многопрофильны...,профессионализм гк базис удаться сформировать ...,Другое


In [293]:
df = df_vacancies
city_filter = 'Москва'
desired_salary_filter = 50000
responsibilities_filter = 'Аналитик'


df_filtered = df[(df['адрес'] == city_filter) &
                 (df['Low_salary'] >= desired_salary_filter) &
                 (df['название вакансии'].str.contains(responsibilities_filter, case=False))]

In [294]:
df_filtered = df_filtered[['описание_unch','адрес','Duties_unch','High_salary','тип занятости','название вакансии','описание вакансии','Skills_unch','URL','company_sphere']]


In [295]:
from gensim.models import Word2Vec
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

vacancy_descriptions = [desc.split() for desc in df_filtered['описание_unch']]
word2vec_model = Word2Vec(sentences=vacancy_descriptions, vector_size=10000, window=5, min_count=1, sg=2)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [296]:
from gensim.models import Word2Vec
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

vacancy_descriptions = [desc.split() for desc in df_filtered['описание_unch']]
word2vec_model = Word2Vec(sentences=vacancy_descriptions, vector_size=10000, window=5, min_count=1, sg=2)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
def recommendations(resume_text, skills, word2vec_model=word2vec_model, bert_model=bert_model, bert_tokenizer=bert_tokenizer):
    resume_embeddings = [word2vec_model.wv[word] for word in resume_text.split() if word in word2vec_model.wv]
    skills_embeddings = [word2vec_model.wv[word] for word in skills.split() if word in word2vec_model.wv]

    resume_mean = np.mean(resume_embeddings, axis=0) if resume_embeddings else np.zeros(word2vec_model.vector_size)
    skills_mean = np.mean(skills_embeddings, axis=0) if skills_embeddings else np.zeros(word2vec_model.vector_size)

    # Convert resume text and skills to BERT embeddings
    resume_bert_input = bert_tokenizer(resume_text, padding=True, truncation=True, return_tensors='pt')
    skills_bert_input = bert_tokenizer(skills, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        resume_output = bert_model(**resume_bert_input).last_hidden_state.mean(dim=1)
        skills_output = bert_model(**skills_bert_input).last_hidden_state.mean(dim=1)

    # Compute the cosine similarity between the embeddings
    resume_similarities_word2vec = [np.dot(word2vec_model.wv[key], resume_mean) for key in word2vec_model.wv.key_to_index.keys() if key in word2vec_model.wv.key_to_index]
    skills_similarities_word2vec = [np.dot(word2vec_model.wv[key], skills_mean) for key in word2vec_model.wv.key_to_index.keys() if key in word2vec_model.wv.key_to_index]

    resume_similarities_bert = torch.nn.functional.cosine_similarity(resume_output, resume_output).item()
    skills_similarities_bert = torch.nn.functional.cosine_similarity(skills_output, skills_output).item()

    # Combine the similarities using a weighted average
    resume_similarities = [(a + resume_similarities_bert) / 2 for a in resume_similarities_word2vec]
    skills_similarities = [(b + skills_similarities_bert) / 2 for b in skills_similarities_word2vec]

    # Sort the similarities in descending order
    resume_similarity_scores = sorted(enumerate(resume_similarities), key=lambda x: x[1], reverse=True)
    skills_similarity_scores = sorted(enumerate(skills_similarities), key=lambda x: x[1], reverse=True)

    # Get the top 5 most similar vacancies for both resume and skills
    resume_similarity_scores = resume_similarity_scores[:5]
    skills_similarity_scores = skills_similarity_scores[:5]

    # Combine and return the top 5 most similar vacancies
    similarity_scores_combined = [(a + b) for a, b in zip(resume_similarity_scores, skills_similarity_scores)]
    vacancy_index_combined = [i[0] for i in similarity_scores_combined]

    acceptance_prob_combined = [i[1]*1.12 for i in similarity_scores_combined]

    return df_filtered[['название вакансии', 'описание вакансии','URL']].iloc[vacancy_index_combined], acceptance_prob_combined

recommendations(' В рамках работы занимаюсь: проведением A/B тестов маркетинговых кампаний, исследованием в области клиентского опыта', 'SQL  маркетинг')


(   название вакансии                                  описание вакансии  \
 25          Аналитик  Добрый день Уважаемый соискатель! Мы динамично...   
 29          Аналитик  Обязанности: Аналитическая поддержка департаме...   
 33          Аналитик  Обязанности: Координация проекта (сроков испол...   
 35          Аналитик  Инвестиционная группа КРИГА – динамично растущ...   
 49  Стартап-Аналитик  Обязанности: • Написание бизнес-планов и созда...   
 
                                                   URL  
 25  https://murmansk.hh.ru/vacancy/88848915?from=v...  
 29  https://murmansk.hh.ru/vacancy/89450342?from=v...  
 33  https://murmansk.hh.ru/vacancy/89831486?from=v...  
 35  https://murmansk.hh.ru/vacancy/89662014?from=v...  
 49  https://murmansk.hh.ru/vacancy/89516749?from=v...  ,
 [0.56, 0.56, 0.56, 0.56, 0.56])

Добавление модели Маккола в рекомендательную сеть

In [297]:
file_path = "ratings_df.xlsx"
df_ratings = pd.read_excel(file_path)
df_ratings.head(2)

,id_resume,id_vacancy,rating
0,1,3856,1
1,1,1242,1


In [298]:
df_merged = pd.merge(df_vacancies, df_ratings, on='id_vacancy', how='outer').fillna(0)
alpha = df_merged.groupby('company_sphere')['rating'].agg(lambda x: sum(x > 1) / len(x))
print(alpha)

company_sphere
IT                       0.117409
Банки IT финансы         0.167135
Государственная сфера    0.146611
Другое                   0.137500
ИП                       0.113350
Инженерно-техническая    0.129797
Консалтинг               0.058824
Розничная торговля       0.052632
Телеком                  0.135135
Торговля                 0.158402
Туризм                   0.095238
Юридические фирмы        0.000000
Name: rating, dtype: float64


In [299]:
df_merged['betta'] = df_merged['company_sphere'].apply(lambda x: 0.167135 if x == 'Банки IT финансы' else (
                                                  0.117409 if x == 'IT' else (
                                                  0.113350 if x == 'ИП' else (
                                                  0.129797 if x == 'Инженерно-техническая' else (
                                                  0.137500 if x == 'Другое' else (
                                                  0.052632 if x == 'Розничная торговля' else (
                                                  0.058824 if x == 'Консалтинг' else (
                                                  0.135135 if x == 'Телеком' else (
                                                  0.158402 if x == 'Торговля' else 0
)))))))))

In [300]:
def lognormal_draws(n=len(df_merged['company_sphere']), μ=np.mean(df_merged['Low_salary']), σ=50000, seed=1234):
    np.random.seed(seed)
    z = np.random.randn(n)
    w_draws = np.exp(μ + σ * z)
    return w_draws

In [301]:
def compute_reservation_wage(mcm):
    """
    Computes the reservation wage of an instance of the McCall model
    by finding the smallest w such that v(w) >= h.

    If no such w exists, then w_bar is set to np.inf.
    """
    u = lambda x: np.log(x/2)

    v, d = solve_model(mcm)
    h = u(mcm.c) + mcm.β * d

    w_bar = np.inf
    for i, wage in enumerate(mcm.w_grid):
        if v[i] > h:
            w_bar = wage
            break

    return w_bar

In [302]:
def lognormal_draws(n=len(df_merged['company_sphere']), μ=np.mean(df_merged['Low_salary']), σ=0.009, seed=1234):
    z = df_merged['betta']
    w_draws = np.exp(μ + σ * z)
    return w_draws

class McCallModelContinuous:
    def __init__(self, c=1, α=0.1, β=0.96, grid_min=1e-10, grid_max=5, grid_size=len(df_merged), w_draws=None, σ=5000):
        self.c, self.α, self.β = c, α, β
        self.w_grid = np.linspace(grid_min, grid_max, grid_size)
      
        if w_draws is None:
            self.w_draws = lognormal_draws(len(self.w_grid), np.mean(self.w_grid), σ)
        else:
            self.w_draws = w_draws
    
    def update(self, mcm):
        c, α, β = self.c, self.α, self.β
        w = self.w_grid
        u = lambda x: np.log(x)

        vf = lambda x: np.interp(x, w, mcm)

        d_new = np.mean(np.maximum(vf(self.w_draws), u(c) + β * mcm))

        v_new = u(w) + β * ((1 - α) * mcm + α * d_new)

        return v_new, d_new

def compute_reservation_wage(mcm):
    u = lambda x: np.log(x/2)

    v, d = mcm.update(mcm)
    h = u(mcm.c) + mcm.β * d

    w_bar = 0
    for i, wage in enumerate(mcm.w_grid):
        if v[i] >= h:
            w_bar = wage
            break

    return w_bar

In [303]:
# Создаем экземпляр модели и вычисляем резервные зарплаты
df_merged['reservation_wage'] = np.random.normal(df_merged['Low_salary'].mean(), 20000 * df_merged['betta'] * 0.96, len(df_merged))

In [304]:

df= df_merged
city_filter = 'Москва'
desired_salary_filter = 50000
responsibilities_filter = 'Аналитик'

df_filtered = df[(df['адрес'] == city_filter) &
                 (df['Low_salary'] >= desired_salary_filter) &
                 (df['название вакансии'].str.contains(responsibilities_filter, case=False))]
vacancy_descriptions = [str(desc).split() for desc in df_filtered['описание_unch']]
word2vec_model = Word2Vec(sentences=vacancy_descriptions, vector_size=10000, window=5, min_count=1, sg=2)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [305]:
from gensim.models import Word2Vec
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
df_filtered = df_merged
def recommendations(resume_text, skills, word2vec_model=word2vec_model, bert_model=bert_model, bert_tokenizer=bert_tokenizer):
    resume_embeddings = [word2vec_model.wv[word] for word in resume_text.split() if word in word2vec_model.wv]
    skills_embeddings = [word2vec_model.wv[word] for word in skills.split() if word in word2vec_model.wv]

    resume_mean = np.mean(resume_embeddings, axis=0) if resume_embeddings else np.zeros(word2vec_model.vector_size)
    skills_mean = np.mean(skills_embeddings, axis=0) if skills_embeddings else np.zeros(word2vec_model.vector_size)

    # Convert resume text and skills to BERT embeddings
    resume_bert_input = bert_tokenizer(resume_text, padding=True, truncation=True, return_tensors='pt')
    skills_bert_input = bert_tokenizer(skills, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        resume_output = bert_model(**resume_bert_input).last_hidden_state.mean(dim=1)
        skills_output = bert_model(**skills_bert_input).last_hidden_state.mean(dim=1)

    # Compute the cosine similarity between the embeddings
    resume_similarities_word2vec = [np.dot(word2vec_model.wv[key], resume_mean) for key in word2vec_model.wv.key_to_index.keys() if key in word2vec_model.wv.key_to_index]
    skills_similarities_word2vec = [np.dot(word2vec_model.wv[key], skills_mean) for key in word2vec_model.wv.key_to_index.keys() if key in word2vec_model.wv.key_to_index]

    resume_similarities_bert = torch.nn.functional.cosine_similarity(resume_output, resume_output).item()
    skills_similarities_bert = torch.nn.functional.cosine_similarity(skills_output, skills_output).item()

    # Combine the similarities using a weighted average
    resume_similarities = [(a + resume_similarities_bert) / 2 for a in resume_similarities_word2vec]
    skills_similarities = [(b + skills_similarities_bert) / 2 for b in skills_similarities_word2vec]
    # Calculate the MacCallum probability
    macallum_probabilities = 1 - (df_filtered['betta'].values / (df_filtered['Low_salary'].values / df_filtered['reservation_wage'].values * np.mean(resume_similarities)))

    # Sort the MacCallum probabilities in descending order
    macallum_scores = sorted(enumerate(macallum_probabilities), key=lambda x: x[1], reverse=True)
    resume_similarity_scores = sorted(enumerate(resume_similarities), key=lambda x: x[1], reverse=True)
    skills_similarity_scores = sorted(enumerate(skills_similarities), key=lambda x: x[1], reverse=True)

    # Get the top 5 most similar vacancies for both resume and skills
    resume_similarity_scores = resume_similarity_scores[:5]
    skills_similarity_scores = skills_similarity_scores[:5]
    macallum_scores = macallum_scores[:5]
    df_filtered['macallum_scores']=macallum_probabilities

    # Combine and return the top 5 most similar vacancies
    similarity_scores_combined = [(a + b) for a, b in zip(resume_similarity_scores, skills_similarity_scores)]
    vacancy_index_combined = [i[0] for i in similarity_scores_combined]
    top_vacancy_index_macallum = [i[0] for i in macallum_scores]
    acceptance_prob_combined = [i[1]*1.12 for i in similarity_scores_combined]
    top_vacancy_prob_macallum = [i[1] for i in macallum_scores]

    return df_filtered[['название вакансии', 'описание вакансии','URL','macallum_scores']].iloc[vacancy_index_combined]
t = recommendations(' В рамках работы занимаюсь: проведением A/B тестов маркетинговых кампаний, исследованием в области клиентского опыта', 'g')


C:\Users\Victoria\AppData\Local\Temp/ipykernel_4700/2195022121.py:32: RuntimeWarning: divide by zero encountered in true_divide
  macallum_probabilities = 1 - (df_filtered['betta'].values / (df_filtered['Low_salary'].values / df_filtered['reservation_wage'].values * np.mean(resume_similarities)))
C:\Users\Victoria\AppData\Local\Temp/ipykernel_4700/2195022121.py:32: RuntimeWarning: invalid value encountered in true_divide
  macallum_probabilities = 1 - (df_filtered['betta'].values / (df_filtered['Low_salary'].values / df_filtered['reservation_wage'].values * np.mean(resume_similarities)))


In [1]:
df_filtered.info()

NameError: name 'df_filtered' is not defined

In [307]:
import pandas as pd
from tkinter.ttk import Combobox
import pandas as pd
import numpy as np
from PIL import ImageTk, Image
import codecs
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import time
from tkinter.ttk import Combobox
import pandas as pd
import numpy as np
from PIL import ImageTk, Image
import codecs
import torch
import tkinter as tk
from tkinter import messagebox, filedialog
import time
from tkinter.ttk import Combobox
import pandas as pd
import numpy as np
from PIL import ImageTk, Image
import codecs
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import time
from tkinter.ttk import Combobox
import pandas as pd
import numpy as np
from PIL import ImageTk, Image
import codecs
import torch

win = tk.Tk()
win.geometry('400x400')
win.title("Job Recommender")
entries = []
for i, label in enumerate(labels):
    tk.Label(win, text=label).grid(row=i, column=0)
    entry = tk.Entry(win)
    entry.grid(row=i, column=1)
    entries.append(entry)

def upload_resume():
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if file_path:
        with codecs.open(file_path, 'r', encoding='utf-8') as file:
            resume_text = file.read()
        if len(entries) > 1:  # Check if entries list has at least two elements
            entries[1].insert(0, resume_text)
        else:
            messagebox.showerror("Index Error", "Entries list index out of bounds.")

upload_button = tk.Button(win, text="Upload Resume", command=upload_resume)
upload_button.grid(row=1, column=2)

start_button = tk.Button(win, text="Start", command=lambda: start_recommendation(entries))
start_button.grid(row=len(labels), column=1)
df_filtered = df_filtered.loc[:,~df_filtered.columns.duplicated()]
df = df.loc[:,~df.columns.duplicated()]

def recommendations(resume_text, skills, word2vec_model=word2vec_model, bert_model=bert_model, bert_tokenizer=bert_tokenizer):
    resume_embeddings = [word2vec_model.wv[word] for word in resume_text.split() if word in word2vec_model.wv]
    skills_embeddings = [word2vec_model.wv[word] for word in skills.split() if word in word2vec_model.wv]

    resume_mean = np.mean(resume_embeddings, axis=0) if resume_embeddings else np.zeros(word2vec_model.vector_size)
    skills_mean = np.mean(skills_embeddings, axis=0) if skills_embeddings else np.zeros(word2vec_model.vector_size)

    # Convert resume text and skills to BERT embeddings
    resume_bert_input = bert_tokenizer(resume_text, padding=True, truncation=True, return_tensors='pt')
    skills_bert_input = bert_tokenizer(skills, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        resume_output = bert_model(**resume_bert_input).last_hidden_state.mean(dim=1)
        skills_output = bert_model(**skills_bert_input).last_hidden_state.mean(dim=1)

    # Compute the cosine similarity between the embeddings
    resume_similarities_word2vec = [np.dot(word2vec_model.wv[key], resume_mean) for key in word2vec_model.wv.key_to_index.keys() if key in word2vec_model.wv.key_to_index]
    skills_similarities_word2vec = [np.dot(word2vec_model.wv[key], skills_mean) for key in word2vec_model.wv.key_to_index.keys() if key in word2vec_model.wv.key_to_index]

    resume_similarities_bert = torch.nn.functional.cosine_similarity(resume_output, resume_output).item()
    skills_similarities_bert = torch.nn.functional.cosine_similarity(skills_output, skills_output).item()

    # Combine the similarities using a weighted average
    resume_similarities = [(a + resume_similarities_bert) / 2 for a in resume_similarities_word2vec]
    skills_similarities = [(b + skills_similarities_bert) / 2 for b in skills_similarities_word2vec]
    # Calculate the MacCallum probability
    
    # Sort the MacCallum probabilities in descending order
    resume_similarity_scores = sorted(enumerate(resume_similarities), key=lambda x: x[1], reverse=True)
    skills_similarity_scores = sorted(enumerate(skills_similarities), key=lambda x: x[1], reverse=True)

    # Get the top 5 most similar vacancies for both resume and skills
    resume_similarity_scores = resume_similarity_scores[:5]
    skills_similarity_scores = skills_similarity_scores[:5]

    # Combine and return the top 5 most similar vacancies
    similarity_scores_combined = [(a + b) for a, b in zip(resume_similarity_scores, skills_similarity_scores)]
    vacancy_index_combined = [i[0] for i in similarity_scores_combined]
    acceptance_prob_combined = [i[1]*1.12 for i in similarity_scores_combined]

    return df_filtered[['название вакансии', 'описание вакансии','URL','macallum_scores']].iloc[vacancy_index_combined]                           
    pass
                                
def export_recommendations(recommended_jobs):
    file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
    if file_path:
        recommended_jobs.to_excel(file_path, index=False)

def start_recommendation(entries):
    global df_filtered
    user_input = [entry.get() for entry in entries]

    if len(user_input) < 7:
        messagebox.showerror("Input Error", "Please fill in all required fields.")
        return

    city_filter = user_input[3]
    salary = int(user_input[4])
    Position = user_input[0]
    Employment_Type = user_input[5]
    Experience = int(user_input[6])

    if city_filter and salary and Position and Employment_Type:
        df_filtered = df[(df['адрес'] == city_filter) & 
                         (df['Low_salary'] >= salary) & 
                         (df['experience'] >= Experience) &
                         (df['experience'] <= Experience+1) &
                         (df['название вакансии'].str.contains(Position, case=False))]
        df_filtered = df_filtered[['описание_unch', 'адрес',  'Low_salary', 'тип занятости', 'название вакансии', 'описание вакансии', 'Skills_unch', 'URL','experience','betta','reservation_wage','Low_salary','macallum_scores']]
        
        # Call the recommendations function and display results
        recommended_jobs = recommendations(user_input[1], user_input[2])
        recommended_jobs['описание вакансии'] = recommended_jobs['описание вакансии'].str.slice(0, 600) + '...'
        recommended_jobs = recommended_jobs[['название вакансии', 'описание вакансии', 'URL','macallum_scores']].head(5)

        # Export recommendations to Excel
        export_recommendations(recommended_jobs)

        new_win = tk.Toplevel(win)
        table = ttk.Treeview(new_win, columns=("название вакансии", "описание вакансии", "URL","macallum_scores"), show="headings")
        table.heading("название вакансии", text="название вакансии")
        table.heading("описание вакансии", text="описание вакансии")
        table.heading("URL", text="ссылка на вакансию")
        table.heading("macallum_scores", text="Вероятность")
        for index, row in recommended_jobs.iterrows():
            table.insert("", "end", values=(row['название вакансии'], row['описание вакансии'], row['URL'], row['macallum_scores']))
        table.pack()
        table.column("название вакансии", width=200)
        table.column("описание вакансии", width=400)
        table.column("URL", width=800)
        table.column("Вероятность", width=800)
    else:
        messagebox.showerror("Input Error", "Please provide valid input for all fields.")    
        return
win.mainloop()

In [262]:
df_merged['Low_salary'].head()

0    70000
1    70000
2    70000
3    70000
4    80000
Name: Low_salary, dtype: int64

In [309]:
labels

['Position',
 'Your resume:',
 'Skills',
 'City',
 'Desired Salary (from)',
 'Employment_Type',
 'Experience']